In [17]:
from gymnasium.spaces import Discrete, Dict, Box,Space
print(Box(low=0,high=1,shape=(2,)).sample())
print(Space((2,),dtype=list))


[0.9602529  0.07039406]


NotImplementedError: 

In [11]:
import gymnasium as gym
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.env.multi_agent_env import  MultiAgentEnv

class MyEnv(MultiAgentEnv):
    def __init__(self, env_config):
        self.MOVES = ['mossa1', 'mossa2','mossa3']

        # Questa è la truncation cosi esce per non girare all'infinito
        self.NUM_ITERS = 20

        # Mappa che in base all'azione eseguita mi da costo, impatto, ecc dell'azione
        self.REWARD_MAP = {
            'mossa1': (1, 0, 0),
            'mossa2': (2, 0, 0),
            'mossa3': (3, 0, 0)
        }

        # per la funzione di reward
        self.wt = 0.5
        self.wc = 0.5
        self.wi = 0.5
        self.tMax = 100
        self.cMax = 100
        self.possible_agents = ['attaccante','difensore']
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )
        # per ogni agente mette le azioni ovvero tante quante le mosse
        # Discrete lo usa per generare uno random con sample()
        self.action_space = {agent: [0,1,2] for agent in self.possible_agents}
        self.observation_space = {
            agent: dict(
                {
                    # observation dovrebbe avere i parametri dello stato,
                    # è lo spazio e lo stato è dato dalla config dei suoi parametri
                    # ho preso 2 dove mossa 1 accende spegne il 1 e mossa due uguale
                    # Box mi genera una lista e mi controlla nel limiti low and high 
                    'observation': [False,False,False],

                    # stessa dimensione delle mosse per selezionare mosse non selezionabili
                    # 2 mosse
                    # Box(low=0,high=1,shape=(2,),dtype=bool)
                    #"action_mask": [True,True]
                }
            )
            for agent in self.possible_agents
        }
        
    def reset(self, seed, options):
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
############################################### QUando termina a me ? ###################################################        
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        #self.state = {agent: NONE for agent in self.agents}
        #self.observations = {agent: NONE for agent in self.agents}

        # credo serve per arrestare
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.reset()
        return self.observation_space,self.infos
    
    def step(self, action):
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            print('ECCOOOO:',self.agents)
            return

        agent = self.agent_selection
        print('Agente in azione:',agent)

        reward = self.REWARD_MAP[self.MOVES[action]]
        valReward = -self.wt*(reward[0]/self.tMax)-self.wc*(reward[1]/self.cMax)-self.wi*(1) 
        #rewardInv = -valReward
        print('Reward:',valReward)
        if self.agent_selection == 'attaccante':
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (valReward,0)
        else:
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (0,valReward)
        
        print('Prima della mossa:',self.observation_space)
        if action == 0:
            self.observation_space[self.agent_selection]['observation'][action]=True
        elif action == 1:
            self.observation_space[self.agent_selection]['observation'][0]=True
            self.observation_space[self.agent_selection]['observation'][action]=True
        elif action == 2:
            self.observation_space[self.agent_selection]['observation'][0]=True
            self.observation_space[self.agent_selection]['observation'][1]=True
            self.observation_space[self.agent_selection]['observation'][action]=True
        print('Dopo la mossa:',self.observation_space)
        self.num_moves += 1
        # The truncations dictionary must be updated for all players.
        self.truncations = {
            agent: self.num_moves >= self.NUM_ITERS for agent in self.agents
        }
        # se uno degli agenti è in uno stato di tutti True esce 
        self.terminations = {
            agent: True if all(item == True for item in self.observation_space[agent]['observation']) else False for agent in self.agents
        }
        print('Deve terminare?',self.terminations)
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()
        return self.observation_space,self.rewards,self.terminations,self.truncations,self.infos

ray.shutdown()
ray.init()
algo = ppo.PPO(env="MyEnv", config={
    "env_config": {},  # config to pass to env class
})

while True:
    print(algo.train())

2023-09-25 09:24:28,986	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-09-25 09:24:30,429	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-25 09:24:30,429	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='MyEnv', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('MyEnv').build()` instead. This will raise an error in the future!
/home/matteo/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this 

EnvError: The env string you provided ('MyEnv') is:
a) Not a supported/installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For VizDoom support: Install VizDoom
   (https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md) and
   `pip install vizdoomgym`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.env.repeat_after_me_env.RepeatAfterMeEnv`


(RolloutWorker pid=319837) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=319837, ip=10.25.252.249, actor_id=95915dfcc79166651d162b4401000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f950d9678b0>)
(RolloutWorker pid=319837)   File "/home/matteo/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 741, in make
(RolloutWorker pid=319837)     env_spec = _find_spec(id)
(RolloutWorker pid=319837)   File "/home/matteo/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 527, in _find_spec
(RolloutWorker pid=319837)     _check_version_exists(ns, name, version)
(RolloutWorker pid=319837)   File "/home/matteo/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py", line 393, in _check_version_exists
(RolloutWorker pid=319837)     _check_name_exists(ns, name)
(RolloutWorker pid=319837)   File "/home/matteo/.local/lib/python3.10/